In [1]:
import sys
import json
import psycopg2
import db
from pgcopy import CopyManager
from model import Receipt
from config import Settings
from api import Api

In [2]:
api = Api()

In [3]:
r = api.get()

In [8]:
data = r.data.decode('utf-8')

In [9]:
sys.getsizeof(data)/1000000

77.018335

In [10]:
data = json.loads(data)

In [11]:
sys.getsizeof(data)/1000000

0.824456

In [12]:
records = [Receipt(**x) for x in data]

In [13]:
sys.getsizeof(records)/1000000

0.824456

In [7]:
schema = Receipt.schema(by_alias=False)
tbl_name = schema.get('db_table_name')
cols = schema.get('properties')

In [41]:
def sql_col_def(name, props) -> str:
    col_type = props.get('type')
    if col_type == 'string':
        if props.get('format') == 'date-time':
            col_def = f"{name} TIMESTAMP"
        elif props.get('maxLength') is not None:
            col_def = f"{name} VARCHAR({props.get('maxLength')})"
        else:
            col_def = f"{name} VARCHAR(250)"
    elif col_type == 'integer':
        col_def = f"{name} INTEGER"
    
    elif col_type == 'boolean':
        col_def = f"{name} BOOLEAN"
    return col_def
    
sql_columns = ",".join([sql_col_def(*item) for item in cols.items()])
sql_table = f"CREATE TABLE IF NOT EXISTS {tbl_name}({tbl_name}_id SERIAL PRIMARY KEY,{sql_columns})"

CREATE TABLE IF NOT EXISTS bev_receipts(bev_receipts_id SERIAL PRIMARY KEY,taxpayer_number VARCHAR(15),taxpayer_name VARCHAR(100),taxpayer_address VARCHAR(100),taxpayer_city VARCHAR(20),taxpayer_state VARCHAR(2),taxpayer_zip VARCHAR(5),taxpayer_county VARCHAR(3),location_number VARCHAR(15),location_name VARCHAR(100),location_address VARCHAR(100),location_city VARCHAR(20),location_state VARCHAR(2),location_zip VARCHAR(5),location_county VARCHAR(3),inside_city_limits BOOLEAN,tabc_permit_number VARCHAR(10),responsibility_begin TIMESTAMP,responsibility_end TIMESTAMP,obligation_end TIMESTAMP,liquor_receipts INTEGER,wine_receipts INTEGER,beer_receipts INTEGER,cover_charge_receipts INTEGER,total_receipts INTEGER)


In [10]:
create_table = """ CREATE TABLE IF NOT EXISTS bev_receipts(
    bev_receipts_id SERIAL PRIMARY KEY,
    taxpayer_number VARCHAR(15),
    taxpayer_name VARCHAR(100),
    taxpayer_address VARCHAR(100),
    taxpayer_city VARCHAR(20),
    taxpayer_state VARCHAR(2),
    taxpayer_zip VARCHAR(5),
    taxpayer_county VARCHAR(3),
    location_number VARCHAR(15),
    location_name VARCHAR(100),
    location_address VARCHAR(100),
    location_city VARCHAR(20),
    location_state VARCHAR(2),
    location_zip VARCHAR(5),
    location_county VARCHAR(3),
    inside_outside_city_limits VARCHAR(1),
    tabc_permit_number VARCHAR(10),
    responsibility_begin_date TIMESTAMP,
    responsibility_end_date TIMESTAMP,
    obligation_end_date TIMESTAMP,
    liquor_receipts INTEGER,
    wine_receipts INTEGER,
    beer_receipts INTEGER,
    cover_charge_receipts INTEGER,
    total_receipts INTEGER
)  """
conn = psycopg2.connect(DB_CONNECT)
cursor = conn.cursor()
cursor.execute(create_table)
conn.commit()
cursor.close()

In [11]:
table_cols = ('taxpayer_number',
 'taxpayer_name',
 'taxpayer_address',
 'taxpayer_city',
 'taxpayer_state',
 'taxpayer_zip',
 'taxpayer_county',
 'location_number',
 'location_name',
 'location_address',
 'location_city',
 'location_state',
 'location_zip',
 'location_county',
 'inside_outside_city_limits',
 'tabc_permit_number',
 'responsibility_begin_date',
 'responsibility_end_date',
 'obligation_end_date',
 'liquor_receipts',
 'wine_receipts',
 'beer_receipts',
 'cover_charge_receipts',
 'total_receipts')

In [12]:
conn = psycopg2.connect(DB_CONNECT)
mgr = CopyManager(conn, 'bev_receipt', table_cols)
mgr.copy([tuple(record.dict().values()) for record in records])
conn.commit()

In [55]:
tuple(x.dict().values())

('17509743203',
 'FIREBIRD DOWNTOWN, INC.',
 '1845 WOODALL RODGERS FWY STE 1100',
 'DALLAS',
 'TX',
 '75201',
 '57',
 '1',
 'EL FENIX RESTAURANT',
 '1601 MCKINNEY AVE',
 'DALLAS',
 'TX',
 '75202',
 '57',
 'Y',
 'MB061420',
 datetime.datetime(1993, 12, 1, 0, 0),
 None,
 datetime.datetime(2007, 1, 31, 0, 0),
 31065,
 1261,
 11020,
 0,
 43346)

In [26]:
Receipt.

<cyfunction BaseModel.dict at 0x7fa62e154380>